# Sujet 2: Modélisation des interdépendances

**Quel est l’impact d’une baisse de la capacité de LNG sur le système en général et en particulier sur
le système électrique ?**

Pour aller plus loin : modéliser le stockage gaz

**Authors**: Lucas DEFRANCE, Quentin DEHAENE, Jules PAINDESSOUS--POETE

## Imports

In [2]:
# Necessary imports
using JuMP
using XLSX
using DataFrames
#use the solver you want
using HiGHS
using CSV
using Statistics

In [3]:
# Define global variables
data_file_zootopia = "data/TP_empilement.xlsx";
data_file = "data/Donnees_etude_de_cas_ETE305.xlsx";

In [4]:
# Extract data for power mix
data_ranges = ["A2:A24", "D2:D24", "E2:E24", "F2:F24", "G2:G24", "H2:H24"]
column_names = [:plant_name, :nb_units, :pmax_per_unit, :pmin_per_unit, :dmin, :costs]
data_types = [String, Int, Float64, Float64, Int, Float64]

# Read data from file
df = DataFrame()
for (col_name, range, data_type) in zip(column_names, data_ranges, data_types)
    df[!, col_name] = data_type.(vec(XLSX.readdata(data_file, "Parc électrique", range)))
end

# Generate the plant_type vector using array comprehension
plant_type = [
    i in [14, 15] ? "thermal_fatal" :
    i == 18 ? "hydro_fatal" :
    i == 19 ? "hydro_lake" :
    i == 20 ? "hydro_step" :
    i in [21, 22] ? "wind" :
    i == 23 ? "solar" :
    "thermal" for i in 1:size(df, 1)
]

# Append the vector as a new column to the DataFrame
df[!, :plant_type] = plant_type

println(df)


In [5]:
# Initialize arrays to store values for non-fatal thermal clusters
names_th, costs_th, Pmin_th, Pmax_th, dmin_th = [], [], [], [], []

# Filter the DataFrame based on the plant_type column
df_thermal = filter(row -> row.plant_type == "thermal", df)

# Loop through each row of the DataFrame
for row in eachrow(df_thermal)
    # Iterate over each plant name and unit number in the row
    for i in 1:row.nb_units
        # Construct the complete plant name with unit number and append it to names_th
        push!(names_th, "$(row.plant_name)-$i")
        
        # Append other values to respective arrays
        push!(costs_th, row.costs)
        push!(Pmin_th, row.pmin_per_unit)
        push!(Pmax_th, row.pmax_per_unit)
        push!(dmin_th, row.dmin)
    end
end

# Convert the vectors to 2D arrays
costs_th, Pmin_th, Pmax_th, dmin_th = hcat(costs_th), hcat(Pmin_th), hcat(Pmax_th), hcat(dmin_th)

# Create a dictionary to map indices to plant names
Nth = length(costs_th)
dict_th = Dict(i => names_th[i] for i in 1:length(names_th))

# Check the value associated with index 1 in dict_th
dict_th[1]



23×7 DataFrame
 Row │ plant_name    nb_units  pmax_per_unit  pmin_per_unit  dmin   costs    plant_type    
     │ String        Int64     Float64        Float64        Int64  Float64  String        
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │ Iconuc               2          900.0          300.0     24     12.0  thermal
   2 │ Tabarnuc             2          900.0          300.0     24     12.0  thermal
   3 │ NucPlusUltra         2         1300.0          330.0     24     12.0  thermal
   4 │  Gazby               1          390.0          135.0      4     40.0  thermal
   5 │  Pégaz               2          394.0          137.0      4     40.0  thermal
   6 │  Samagaz             1          430.0          151.0      4     40.0  thermal
   7 │  Omaïgaz             2          430.0          151.0      4     40.0  thermal
   8 │  Gastafiore          1          430.0          151.0      4     40.0  thermal
   9 │ Igaznodon            2

"Iconuc-1"

In [6]:
#data for hydro reservoir
Nhy = 1 #number of hydro generation units
Pmin_hy = zeros(Nhy);
Pmax_hy = df[19,:pmax_per_unit] *ones(Nhy); #MW
e_hy = XLSX.readdata(data_file, "Détails historique hydro", "B2:B13");
# Calculate the mean stock level
weekly_mean_e_hy = repeat([sum(e_hy) / 52], Nhy)
costs_hy = df[19,:costs]*ones(Nhy); #MWh

#data for STEP
Pmax_STEP = df[20,:pmax_per_unit] #MW
rSTEP = 0.75
max_stock_STEP = 201600; #MWh

In [7]:
# We choose to use run 52 simulations of 1 week each
Tp = 24*7

168

In [19]:
#Import gaz data

V_max_gaz=XLSX.readdata(data_file, "Données gaz", "K4"); #en MwH attention
In_max_gaz=XLSX.readdata(data_file, "Données gaz", "L4"); #Mw/j
Out_max_gaz=XLSX.readdata(data_file, "Données gaz", "M4");#Mw/j
Niv_min_DtD=XLSX.readdata(data_file, "Données gaz", "P5:P368"); #A multiplier par Vmax
Niv_max_DtD=XLSX.readdata(data_file, "Données gaz", "R5:R368");#A multiplier par Vmax
Load_gaz_DtD=XLSX.readdata(data_file, "Données gaz", "C5:C370")#Mwh/j

Niv_min_HtH = [value*V_max_gaz for value in Niv_min_DtD for _ in 1:24];
Niv_max_HtH = [value*V_max_gaz for value in Niv_max_DtD for _ in 1:24];
Coeff_jour=[0.028,0.029,0.031,0.034,0.038,0.048,0.069,0.067,0.066,0.06,0.053,0.047,0.04,0.037,0.033,0.032,0.036,0.039,0.041,0.04,0.038,0.037,0.031,0.027]
Load_gaz_HtH=[value*Coeff_jour[j] for value in Load_gaz_DtD for j in 1:24];
println(Load_gaz_HtH)

coeff_import = 0.5; #coefficient d'importation à modifier si on stop la Russie
#25% d'imports russes

[4019.06034585664, 4162.59821535152, 4449.67395434128, 4880.28756282592, 5454.43904080544, 6889.81773575424, 9904.112995146721, 9617.03725615696, 9473.49938666208, 8612.2721696928, 7607.507083228639, 6746.27986625936, 5741.5147797952, 5310.901171310559, 4736.74969333104, 4593.21182383616, 5167.36330181568, 5597.97691030032, 5885.05264929008, 5741.5147797952, 5454.43904080544, 5310.901171310559, 4449.67395434128, 3875.52247636176, 4008.821138124572, 4151.993321629021, 4438.337688637919, 4867.8542391512665, 5440.542973169062, 6872.264808213552, 9878.880661806981, 9592.536294798083, 9449.364111293635, 8590.33101026694, 7588.125725735797, 6729.092624709103, 5726.88734017796, 5297.370789664613, 4724.682055646817, 4581.509872142368, 5154.198606160164, 5583.715156673511, 5870.059523682409, 5726.88734017796, 5440.542973169062, 5297.370789664613, 4438.337688637919, 3865.648954620123, 3999.3502343593955, 4142.1841713008025, 4427.852045183617, 4856.353856007838, 5427.689603773465, 6856.0289731875

In [9]:
# Extract data for power mix
data_ranges = ["V15:V17", "W15:W17", "X15:X17", "Y15:Y17", "Z15:Z17"]
column_names = [:prod_name, :pmax, :pmin, :dmin, :costs]
data_types = [String, Float64, Float64, Int, Float64]

# Read data from file
df = DataFrame()
for (col_name, range, data_type) in zip(column_names, data_ranges, data_types)
    df[!, col_name] = data_type.(vec(XLSX.readdata(data_file, "Données gaz", range)))
end


println(df)

3×5 DataFrame
 Row │ prod_name          pmax      pmin     dmin   costs   
     │ String             Float64   Float64  Int64  Float64 
─────┼──────────────────────────────────────────────────────
   1 │ Methanisation       6250.0   6000.0     168     24.0
   2 │ Imports GNL        12500.0      0.0       1    150.0
   3 │ Pyrogazeification   2083.33  1708.33    168     25.0


In [14]:
# Initialize arrays to store values for non-fatal thermal clusters
names_gaz, costs_gaz, Pmin_gaz, Pmax_gaz, dmin_gaz = [], [], [], [], []

# Loop through each row of the DataFrame
for row in eachrow(df)
 # Iterate over each plant name and unit number in the row

    # Construct the complete plant name with unit number and append it to names_th
    push!(names_gaz, row.prod_name)
    
    # Append other values to respective arrays
    push!(costs_gaz, row.costs)
    push!(Pmin_gaz, row.pmin)
    push!(Pmax_gaz, row.pmax)
    push!(dmin_gaz, row.dmin)

end

# Convert the vectors to 2D arrays
costs_gaz, Pmin_gaz, Pmax_gaz, dmin_gaz = hcat(costs_gaz), hcat(Pmin_gaz), hcat(Pmax_gaz), hcat(dmin_gaz)

# Create a dictionary to map indices to plant names
Ngaz = length(costs_gaz)
dict_gaz= Dict(i => names_gaz[i] for i in 1:length(names_gaz))

println(Pmax_gaz)

[6250.0; 12500.0; 2083.3333333333335;;]


In [22]:
nb_week_year = 2
UC_continu, stock_STEP_continu, UP_continu, DO_continu = [], [], [], []
UC_continu_gaz, stock_gaz_continu, UP_continu_gaz, DO_continu_gaz = [],0, [], []
sizeUP_gaz=0
Load_gaz=[]
Load_elec_gaz=[]
for w in 1:nb_week_year
    # We start the simulation at Tmin and end it at Tmax
    Tmin = (w-1)*24*7
    Tmax = Tmin + 24*7-1; #optimization by one week intervals
    Tp = 24*9;

      
    
    #data for load and fatal generation
    jour=XLSX.readdata(data_file_zootopia, "TP", "A$(10+Tmin):B$(Tp+Tmin + 10)");
    ToD=XLSX.readdata(data_file_zootopia, "TP", "B$(10+Tmin):B$(Tp+Tmin + 10)");
    load = XLSX.readdata(data_file_zootopia, "TP", "C$(10+Tmin):C$(Tp+Tmin + 10)");
    wind = XLSX.readdata(data_file_zootopia, "TP", "D$(10+Tmin):D$(Tp+Tmin + 10)");
    solar = XLSX.readdata(data_file_zootopia, "TP", "E$(10+Tmin):E$(Tp+Tmin + 10)");
    hydro_fatal = XLSX.readdata(data_file_zootopia, "TP", "F$(10+Tmin):F$(Tp+Tmin + 10)");
    thermal_fatal = XLSX.readdata(data_file_zootopia, "TP", "G$(10+Tmin):G$(Tp+Tmin + 10)");
    #total of RES
    Pres = wind + solar + hydro_fatal + thermal_fatal;
    # println(jour)
    
    #costs
    cth = repeat(costs_th', Tp) #cost of thermal generation €/MWh
    chy = repeat(costs_hy', Tp) #cost of hydro generation €/MWh
    cuns = 5000*ones(Tp) #cost of unsupplied energy €/MWh
    cexc = 0*ones(Tp); #cost of in excess energy €/MWh


    #############################
    #create the optimization model
    #############################
    model = Model(HiGHS.Optimizer)

    #############################
    #define the variables
    #############################
    #thermal generation variables
    @variable(model, Pth[1:Tp,1:Nth] >= 0)
    @variable(model, UCth[1:Tp,1:Nth], Bin)
    @variable(model, UPth[1:Tp,1:Nth], Bin)
    @variable(model, DOth[1:Tp,1:Nth], Bin)
    #hydro generation variables
    @variable(model, Phy[1:Tp,1:Nhy] >= 0)
    #unsupplied energy variables
    @variable(model, Puns[1:Tp] >= 0)
    #in excess energy variables
    @variable(model, Pexc[1:Tp] >= 0)
    #weekly STEP variables
    @variable(model, Pcharge_STEP[1:Tp] >= 0)
    @variable(model, Pdecharge_STEP[1:Tp] >= 0)
    @variable(model, stock_STEP[1:Tp] >= 0)


    #GAZ VARIABLES

    Load_gaz=Load_gaz_HtH[Tmin+1:Tmin+Tp]
    cgaz=repeat(costs_gaz',Tp)

    @variable(model, P_injected_gaz[1:Tp] >= 0)
    @variable(model, P_discharged_gaz[1:Tp] >= 0)
    @variable(model, stock_gaz[1:Tp] >= 0)
    
    @variable(model, 0<=P_gaz[1:Tp,1:Ngaz])
    @variable(model, UC_gaz[1:Tp,1:Ngaz], Bin)
    @variable(model, UP_gaz[1:Tp,1:Ngaz], Bin)
    @variable(model, DO_gaz[1:Tp,1:Ngaz], Bin)

    #unsupplied gaz variables
    @variable(model, Puns_gaz[1:Tp] >= 0)
    #in excess gaz variables
    @variable(model, Pexc_gaz[1:Tp] >= 0)

    cuns_gaz = 5000*ones(Tp) #cost of unsupplied gaz €/MWh
    cexc_gaz = 1*ones(Tp); #cost of in excess gaz €/MWh
    

    #############################
    #define the objective function
    #############################

    @objective(model, Min, sum(Pth.*cth)+sum(Phy.*chy)+Puns'cuns+Pexc'cexc+sum(P_gaz.*cgaz)
    +Puns_gaz'cuns_gaz+Pexc_gaz'cexc_gaz)
    


    #############################
    #define the constraints
    #############################
    #balance constraint
    @constraint(model, balance[t in 1:Tp], sum(Pth[t,g] for g in 1:Nth) + sum(Phy[t,h] for h in 1:Nhy) +
     Pres[t] + Puns[t] - load[t] - Pexc[t] - Pcharge_STEP[t] + Pdecharge_STEP[t]== 0)
    #thermal unit Pmax constraints
    @constraint(model, max_th[t in 1:Tp, g in 1:Nth], Pth[t,g] <= Pmax_th[g]*UCth[t,g])
    #thermal unit Pmin constraints
    @constraint(model, min_th[t in 1:Tp, g in 1:Nth], Pmin_th[g]*UCth[t,g] <= Pth[t,g])


   # contraintes de limitation de variation de la puissance (350 MWh/h 200 MWh CCG)
   @constraint(model, rampenuc_asc1[t in 2:Tp, i in 1:6], Pth[t,i] - Pth[t-1,i] <= 350)
   @constraint(model, rampenuc1_desc[t in 2:Tp, i in 1:6], Pth[t,i] - Pth[t-1,i] >= -350)
   @constraint(model, rampeccg_asc[t in 2:Tp,i in 7:13], Pth[t,i] - Pth[t-1,i] <= 200)
   @constraint(model, rampeccg_desc[t in 2:Tp,i in 7:13], Pth[t,i] - Pth[t-1,i] >= -200)
    #thermal unit Dmin constraints
    for g in 1:Nth
        if (dmin_th[g] > 1)
            @constraint(model, [t in 2:Tp], UCth[t,g]-UCth[t-1,g]==UPth[t,g]-DOth[t,g],  base_name = "fct_th_$g")
            @constraint(model, [t in 1:Tp], UPth[t]+DOth[t]<=1,  base_name = "UPDOth_$g")
            if (w==1)
                @constraint(model, UPth[1,g]==0,  base_name = "iniUPth_$g")
                @constraint(model, DOth[1,g]==0,  base_name = "iniDOth_$g")
            else
                @constraint(model, UCth[1,g]-UC_continu[g]==UPth[1,g]-DOth[1,g])
            end   
            @constraint(model, [t in dmin_th[g]:Tp], UCth[t,g] >= sum(UPth[i,g] for i in (t-dmin_th[g]+1):t))
            @constraint(model, [t in dmin_th[g]:Tp], UCth[t,g] <= 1 - sum(DOth[i,g] for i in (t-dmin_th[g]+1):t))
            if (w==1)
                @constraint(model, [t in 1:dmin_th[g]-1], UCth[t,g] >= sum(UPth[i,g] for i in 1:t))
                @constraint(model, [t in 1:dmin_th[g]-1], UCth[t,g] <= 1-sum(DOth[i,g] for i in 1:t))

            else
                @constraint(model, [t in 1:dmin_th[g]-1], UCth[t,g] >= sum(UPth[i,g] for i in 1:t)+sum(UP_continu[j,g] for j in 25 + t - dmin_th[g]:25), base_name = "dminUPth_$(g)_init_cont")
                @constraint(model, [t in 1:dmin_th[g]-1], UCth[t,g] <= 1-sum(DOth[i,g] for i in 1:t)-sum(DO_continu[j,g] for j in 25 + t - dmin_th[g]:25), base_name = "dminDOth_$(g)_init_cont")
            end
        end
    end 

    
    #hydro unit constraints
    @constraint(model, unit_bounds_hy[t in 1:Tp, h in 1:Nhy], Pmin_hy[h] <= Phy[t,h] <= Pmax_hy[h])
    #hydro stock constraint
    @constraint(model, stock_bounds_hy[h in 1:Nhy], sum(Phy[t,h] for t in 1:Tp) <= weekly_mean_e_hy[h] )

    #weekly STEP
    @constraint(model, max_Pcharge_STEP[t in 1:Tp],Pcharge_STEP[t] <= Pmax_STEP)
    @constraint(model, max_Pdecharge_STEP[t in 1:Tp],Pdecharge_STEP[t] <= Pmax_STEP)

    @constraint(model, stock_STEP_max[t in 1:Tp], stock_STEP[t] <= max_stock_STEP)
    @constraint(model, stock_STEP_evo[t in 2:Tp],stock_STEP[t] == stock_STEP[t-1] + Pcharge_STEP[t]*rSTEP - Pdecharge_STEP[t])
    if (w==1)
        @constraint(model,stock_STEP[1] == 0.15*max_stock_STEP)
    else
        @constraint(model,stock_STEP[1] == stock_STEP_continu + Pcharge_STEP[1]*rSTEP - Pdecharge_STEP[1])
    end


    #GAZ CONSTRAINTS
    @constraint(model, stck_gaz_encad[t in 1:Tp], Niv_min_HtH[t]<=stock_gaz[t]<= Niv_max_HtH[t])
    @constraint(model, stck_gaz[t in 2:Tp], stock_gaz[t] == stock_gaz[t-1]+ P_injected_gaz[t] - P_discharged_gaz[t])
    if (w==1)
        @constraint(model, stock_gaz[1]==0.599*V_max_gaz)#arbitrary choice to start at mean level
       
    else
        @constraint(model, stock_gaz[1]==stock_gaz_continu)
    end   

    #thermal unit Pmax constraints
    @constraint(model, max_gaz_meth[t in 1:Tp], P_gaz[t,1] <= Pmax_gaz[1]*UC_gaz[t,1])
    @constraint(model, max_gaz_gnl[t in 1:Tp], P_gaz[t,2] <= Pmax_gaz[2]*UC_gaz[t,2]*coeff_import)#controlling how much we can import 
    @constraint(model, max_gaz_pyro[t in 1:Tp], P_gaz[t,3] <= Pmax_gaz[3]*UC_gaz[t,3])

    #thermal unit Pmin constraints
    @constraint(model, min_gaz[t in 1:Tp, g in 1:Ngaz], Pmin_gaz[g]*UC_gaz[t,g] <= P_gaz[t,g])

    #gaz in and out constraints
    @constraint(model,inj_gaz[t in 1:Tp],P_injected_gaz[t]<=In_max_gaz/24)#limited by pipe flow
    @constraint(model,out_gaz_max[t in 1:Tp],P_discharged_gaz[t]<=Out_max_gaz/24)

    Load_elec_gaz=[0 for i in 1:Tp]
    (Load_elec_gaz[t]=(Pth[t,7]+Pth[t,8]+Pth[t,9]+Pth[t,10]+Pth[t,11]+Pth[t,12]+Pth[t,13])/0.6 + (Pth[t,14]+Pth[t,15])/0.4 + (Pth[t,16]+Pth[t,17]+Pth[t,18])/0.5 for t in 1:Tp)
    #Pth representent les puissances des centrales gaz qu'il faut donc approvisonner en gaz avec un rendement

    @constraint(model,out_gaz[t in 1:Tp], P_discharged_gaz[t]+ sum(P_gaz[t,g] for g in 1:Ngaz) + Puns_gaz[t]- Pexc_gaz[t]
    -(P_injected_gaz[t]+Load_gaz[t]+Load_elec_gaz[t])==0)
    



    for g in 1:Ngaz
        if (dmin_gaz[g] > 1)
            @constraint(model, [t in 2:Tp], UC_gaz[t,g]-UC_gaz[t-1,g]==UP_gaz[t,g]-DO_gaz[t,g])
            @constraint(model, [t in 1:Tp], UP_gaz[t,g]+DO_gaz[t,g]<=1)
            if (w==1)
                @constraint(model, UP_gaz[1,g]==0)
                @constraint(model, DO_gaz[1,g]==0)
            else
                @constraint(model, UC_gaz[1,g]-UC_continu_gaz[g]==UP_gaz[1,g]-DO_gaz[1,g])
            end   
            @constraint(model, [t in dmin_gaz[g]:Tp], UC_gaz[t,g] >= sum(UP_gaz[i,g] for i in (t-dmin_gaz[g]+1):t))
            @constraint(model, [t in dmin_gaz[g]:Tp], UC_gaz[t,g] <= 1 - sum(DO_gaz[i,g] for i in (t-dmin_gaz[g]+1):t))
            if (w==1)
                @constraint(model, [t in 1:dmin_gaz[g]-1], UC_gaz[t,g] >= sum(UP_gaz[i,g] for i in 1:t))
                @constraint(model, [t in 1:dmin_gaz[g]-1], UC_gaz[t,g] <= 1-sum(DO_gaz[i,g] for i in 1:t))
    
            else
                @constraint(model, [t in 1:dmin_gaz[g]-1], UC_gaz[t,g] >= sum(UP_gaz[i,g] for i in 1:t)+sum(UP_continu_gaz[j,g] for j in sizeUP_gaz + t - dmin_gaz[g]:sizeUP_gaz))
                @constraint(model, [t in 1:dmin_gaz[g]-1], UC_gaz[t,g] <= 1-sum(DO_gaz[i,g] for i in 1:t)-sum(DO_continu_gaz[j,g] for j in sizeUP_gaz + t - dmin_gaz[g]:sizeUP_gaz))
            end
        
        end
    end 
    

    

    #no need to print the model when it is too big
    #solve the model
    optimize!(model)
    
    # Collecting values for each iteration
    UC_continu= value.(UCth[Tp-48, 1:Nth])
    stock_STEP_continu= value(stock_STEP[Tp-48])
    UP_continu= value.(UPth[Tp-72:Tp-48, 1:Nth])
    DO_continu= value.(DOth[Tp-72:Tp-48, 1:Nth])

    UC_continu_gaz= value.(UC_gaz[Tp-48, 1:Ngaz])
    UP_continu_gaz= value.(UP_gaz[Tp-215:Tp-48, 1:Ngaz])
    DO_continu_gaz= value.(DO_gaz[Tp-215:Tp-48, 1:Ngaz])
    stock_gaz_continu= value(stock_gaz[Tp-48])
    sizeUP_gaz=size(UP_continu_gaz,1)
    println(sizeUP_gaz)
    
    #------------------------------
    #Results
    # println("semaine :",w)
    @show termination_status(model)
    # @show objective_value(model)

    #exports results as csv file
    th_gen = value.(Pth)
    hy_gen = value.(Phy)
    STEP_charge = value.(Pcharge_STEP)
    STEP_decharge = value.(Pdecharge_STEP)
    gaz_gen=value.(P_gaz)
    gaz_stock_in=value.(P_injected_gaz)
    gaz_stock_discharged=value.(P_discharged_gaz)
    gaz_uns=value.(Puns_gaz)
    gaz_exec=value.(Pexc_gaz)
    stock_gaz=value.(stock_gaz)
    # new file created
    file_name = string("results_avec_gaz/resultsW",w,".csv")
    touch(file_name)

    # file handling in write mode
    f = open(file_name, "w")

    write(f, "Jour, Heure, Production fatal, Demande, Thermique restant,")
    for i in 1:length(dict_th)
        plant_name = dict_th[i]
        write(f, string(plant_name, ","))
    end
    write(f,"Hydro, STEP pompage, STEP turbinage,")

    write(f, "Demande en CH4,")
    for i in 1:length(dict_gaz)
        plant_name = dict_gaz[i]
        write(f, string(plant_name, ","))
    end
    write(f, "Stock Gaz,Stock Gaz in, Stock Gaz out, P Uns Gaz, P Exc Gaz \n")
   
    

    
    for t in 1:Tp-48
        write(f,'\n')
        write(f, "$(jour[t]), $(ToD[t]),$(Pres[t]), $(load[t]), $(load[t]-Pres[t]),")
        for g in 1:Nth
            write(f, "$(th_gen[t,g]), ")
        end
        for h in 1:Nhy
            write(f, "$(hy_gen[t,h]),")
        end
        write(f, "$(STEP_charge[t]), $(STEP_decharge[t]),")
        write(f,"$(Load_gaz[t]), ")
        for g in 1:Ngaz
            write(f, "$(gaz_gen[t,g]), ")
        
        end
        
        write(f,"$(stock_gaz[t]),$(gaz_stock_in[t]),$(gaz_stock_discharged[t]),$(gaz_uns[t]),$(gaz_exec[t])")
        
    
        
    end
    

    close(f)
end

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
33593 rows, 21342 cols, 246772 nonzeros
23990 rows, 18190 cols, 244737 nonzeros
23661 rows, 17984 cols, 250144 nonzeros

Solving MIP model with:
   23661 rows
   17984 cols (10856 binary, 0 integer, 0 implied int., 7128 continuous)
   250144 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -34256220.44722 inf                  inf        0      0      0         0     3.5s
 R       0       0         0   0.00%   10818991.38462  10818991.38462     0.00%        0      0      0      3596     5.5s

Solving report
  Status            Optimal
  Primal bound      10818991.3846
  Dual bound        10818991.3846
  Gap               0% (tolerance: 0.01%)
  Solution status 

In [12]:
# using CSV
# using DataFrames

# # File paths
# file_paths = []
# for i in 1:52
#     file_name = "results/resultsW$i.csv"
#     push!(file_paths, file_name)
# end

# # Initialize an empty DataFrame to store combined data
# combined_df = DataFrame()

# # Loop through each file path
# for file_path in file_paths
#     # Read the CSV file
#     df = CSV.read(file_path, DataFrame)
    
#     # If it's not the first file, skip the header
#     if nrow(combined_df) > 0
#         df = df[2:end, :]
#     end
    
#     # Append the data to the combined DataFrame
#     append!(combined_df, df)
# end

# # Write the combined DataFrame to a new CSV file
# CSV.write("results/combined_results.csv", combined_df)

